In [2]:
import pandas as pd
import numpy as np
pd.options.display.max_rows=1000
pd.options.display.max_columns=1000
from multiprocessing import Pool

In [3]:
train=pd.read_csv('../train.csv')
test=pd.read_csv('../test_QyjYwdj.csv')
item=pd.read_csv('../item_data.csv')
transaction=pd.read_csv('../customer_transaction_data.csv')
coupon_item_mapping=pd.read_csv('../coupon_item_mapping.csv')
camp=pd.read_csv('../campaign_data.csv')
camp.start_date=pd.to_datetime(camp.start_date,format='%d/%m/%y')
camp.end_date=pd.to_datetime(camp.end_date,format='%d/%m/%y')
camp.sort_values('start_date',inplace=True)
df=pd.concat((train.iloc[:,:-1],test),axis=0)
df=df.merge(camp,on='campaign_id',how='left')
transaction.date=pd.to_datetime(transaction.date)

In [4]:
df.head()

,id,campaign_id,coupon_id,customer_id,campaign_type,start_date,end_date
0,1,13,27,1053,X,2013-05-19,2013-07-05
1,2,13,116,48,X,2013-05-19,2013-07-05
2,6,9,635,205,Y,2013-03-11,2013-04-12
3,7,13,644,1050,X,2013-05-19,2013-07-05
4,9,8,1017,1489,X,2013-02-16,2013-04-05


In [5]:
def get_coupons_for_customer(cust_id):
    return df[df.customer_id==cust_id].coupon_id.unique().tolist()
def get_items_for_coupons(coupon_list):
    return coupon_item_mapping[coupon_item_mapping.coupon_id.isin(coupon_list)].item_id.unique().tolist()
def get_custs_for_coupon(coup_id):
    return df[df.coupon_id==coup_id].customer_id.unique().tolist()
def get_coupons_for_customer_camp(cust_id,camp_id):
    return df[(df.customer_id==cust_id)&(df.campaign_id==camp_id)].coupon_id.unique().tolist()

In [6]:
data_=df.iloc[0]

In [7]:
import datetime

In [8]:
df.head()

,id,campaign_id,coupon_id,customer_id,campaign_type,start_date,end_date
0,1,13,27,1053,X,2013-05-19,2013-07-05
1,2,13,116,48,X,2013-05-19,2013-07-05
2,6,9,635,205,Y,2013-03-11,2013-04-12
3,7,13,644,1050,X,2013-05-19,2013-07-05
4,9,8,1017,1489,X,2013-02-16,2013-04-05


In [9]:
%%time
def get_feat(row):
    data_=row[1]
    #### this is for test_feat_2
    l=get_custs_for_coupon(data_.coupon_id)
    l.remove(data_.customer_id)
    temp=transaction[(transaction.customer_id.isin(l))&\
                     transaction.item_id.isin(get_items_for_coupons([data_.coupon_id]))
                    ]
    
    
    if temp.shape[0]>0:
            feats=[data_.id]+[sum(temp.coupon_discount<0),temp.shape[0],temp.coupon_discount.sum()]
    else:
        feats=[data_['id']]+np.repeat(0,3).tolist()

    return feats
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, df.iterrows())
finally: 
    pool.close()
    pool.join()
    
cols=['id','test1_var2','test2_var2','test3_var2']
test_feat=pd.DataFrame(np.row_stack(data_outputs),columns=cols)
test_feat.to_csv('../test_feat_2.csv',index=False)

CPU times: user 22 s, sys: 1.77 s, total: 23.8 s
Wall time: 1h 8min 28s


In [10]:
%%time
def get_feat(row):
    data_=row[1]
    #### this is for test_feat_2
    temp=transaction[(transaction.customer_id!=data_.customer_id)&\
                     transaction.item_id.isin(get_items_for_coupons([data_.coupon_id]))
                    ]
    
    if temp.shape[0]>0:
            feats=[data_.id]+[sum(temp.coupon_discount<0),temp.shape[0],temp.coupon_discount.sum()]
    else:
        feats=[data_['id']]+np.repeat(0,3).tolist()

    return feats
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, df.iterrows())
finally: 
    pool.close()
    pool.join()
    
cols=['id','test1','test2','test3']
test_feat=pd.DataFrame(np.row_stack(data_outputs),columns=cols)
test_feat.to_csv('../test_feat.csv',index=False)

CPU times: user 19.8 s, sys: 584 ms, total: 20.4 s
Wall time: 11min 23s


In [11]:
%%time
def get_feat(row):
    data_=row[1]
    temp=transaction[(transaction.customer_id.isin(get_custs_for_coupon(data_.coupon_id)))&\
                (transaction.item_id.isin(get_items_for_coupons([data_.coupon_id])))&\
               (transaction.date<data_.start_date)]
    if temp.shape[0]>0:
        a=temp['coupon_discount']/temp['selling_price']
        feats=[data_['campaign_id'],data_['coupon_id']]+[temp.shape[0],   sum(temp['coupon_discount']<0),  sum(temp['coupon_discount']<0)/temp.shape[0],\
                          temp['coupon_discount'].sum(),temp['selling_price'].sum(),
                          
                          temp['coupon_discount'].mean(),temp['selling_price'].mean(),temp['other_discount'].sum(),\
                          temp['other_discount'].mean(),a.mean(),a.sum() 
                         ]
    else:
        feats=[data_['campaign_id'],data_['coupon_id']]+np.repeat(0,11).tolist()

    return feats
temp=df[['coupon_id','start_date','campaign_id']].drop_duplicates()
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, temp.iterrows())
finally: 
    pool.close()
    pool.join()
    
cols=['campaign_id','coupon_id','num_bought_coup_var1','num_discount_availed_coup_var1','frac_discount_availed_coup_var1','sum_discount_coup_var1',
'selling_price_sum_coup_var1',
'coupon_discount_mean_coup_var1','selling_price_mean_coup_var1','other_discount_sum_coup_var1',
      'other_discount_mean_coup_var1','cd_sp_ratio_mean_coup_var1','cd_sp_ratio_sum_coup_var1'
     ]
coup_hist1=pd.DataFrame(np.row_stack(data_outputs),columns=cols)

CPU times: user 208 ms, sys: 148 ms, total: 356 ms
Wall time: 38.1 s


In [12]:
coup_hist1.to_csv('../coup_hist1_v2.csv',index=False)

In [13]:
%%time
def get_feat(row):
    data_=row[1]
    temp=transaction[(transaction.customer_id==data_.customer_id)&\
                (transaction.item_id.isin(get_items_for_coupons(get_coupons_for_customer(data_.customer_id))))&\
               (transaction.date<data_.start_date)]
    if temp.shape[0]>0:
        a=temp['coupon_discount']/temp['selling_price']
        feats=[data_['campaign_id'],data_['customer_id']]+[temp.shape[0],   sum(temp['coupon_discount']<0),  sum(temp['coupon_discount']<0)/temp.shape[0],\
                          temp['coupon_discount'].sum(),temp['selling_price'].sum(),
                          
                          temp['coupon_discount'].mean(),temp['selling_price'].mean(),temp['other_discount'].sum(),\
                          temp['other_discount'].mean(),a.mean(),a.sum() 
                         ]
    else:
        feats=[data_['campaign_id'],data_['customer_id']]+np.repeat(0,11).tolist()

    return feats
temp=df[['customer_id','start_date','campaign_id']].drop_duplicates()
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, temp.iterrows())
finally: 
    pool.close()
    pool.join()
    
cols=['campaign_id','customer_id','num_bought_cust_var1','num_discount_availed_cust_var1','frac_discount_availed_cust_var1','sum_discount_cust_var1',
'selling_price_sum_cust_var1',
'coupon_discount_mean_cust_var1','selling_price_mean_cust_var1','other_discount_sum_cust_var1',
      'other_discount_mean_cust_var1','cd_sp_ratio_mean_cust_var1','cd_sp_ratio_sum_cust_var1'
     ]
cust_coup_hist=pd.DataFrame(np.row_stack(data_outputs),columns=cols)

CPU times: user 1.07 s, sys: 216 ms, total: 1.29 s
Wall time: 4min 49s


In [14]:
cust_coup_hist.head()

,campaign_id,customer_id,num_bought_cust_var1,num_discount_availed_cust_var1,frac_discount_availed_cust_var1,sum_discount_cust_var1,selling_price_sum_cust_var1,coupon_discount_mean_cust_var1,selling_price_mean_cust_var1,other_discount_sum_cust_var1,other_discount_mean_cust_var1,cd_sp_ratio_mean_cust_var1,cd_sp_ratio_sum_cust_var1
0,13.0,1053.0,33.0,0.0,0.000000,0.00,2953.95,0.000000,89.513636,-1145.51,-34.712424,0.000000,0.000000
1,13.0,48.0,47.0,1.0,0.021277,-17.81,5707.71,-0.378936,121.440638,-2027.79,-43.144468,-0.004273,-0.200812
2,9.0,205.0,240.0,14.0,0.058333,-315.22,33351.16,-1.313417,138.963167,-5497.62,-22.906750,-0.014588,-3.501007
3,13.0,1050.0,45.0,0.0,0.000000,0.00,2663.99,0.000000,59.199778,-632.59,-14.057556,0.000000,0.000000
4,8.0,1489.0,61.0,2.0,0.032787,-32.06,7627.26,-0.525574,125.037049,-1886.78,-30.930820,-0.005126,-0.312710


In [15]:
%%time
def get_feat(row):
    data_=row[1]
    temp=transaction[(transaction.customer_id==data_.customer_id)&\
    (transaction.item_id.isin(get_items_for_coupons(get_coupons_for_customer_camp(data_.customer_id,data_.campaign_id))))&\
               (transaction.date<data_.start_date)]
    if temp.shape[0]>0:
        a=temp['coupon_discount']/temp['selling_price']
        feats=[data_['campaign_id'],data_['customer_id']]+[temp.shape[0],   sum(temp['coupon_discount']<0),  sum(temp['coupon_discount']<0)/temp.shape[0],\
                          temp['coupon_discount'].sum(),temp['selling_price'].sum(),
                          
                          temp['coupon_discount'].mean(),temp['selling_price'].mean(),temp['other_discount'].sum(),\
                          temp['other_discount'].mean(),a.mean(),a.sum() 
                         ]
    else:
        feats=[data_['campaign_id'],data_['customer_id']]+np.repeat(0,11).tolist()

    return feats
temp=df[['customer_id','start_date','campaign_id']].drop_duplicates()
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, temp.iterrows())
finally: 
    pool.close()
    pool.join()
    
cols=['campaign_id','customer_id','num_bought_cust_var3','num_discount_availed_cust_var3','frac_discount_availed_cust_var3','sum_discount_cust_var3',
'selling_price_sum_cust_var3',
'coupon_discount_mean_cust_var3','selling_price_mean_cust_var3','other_discount_sum_cust_var3',
      'other_discount_mean_cust_var3','cd_sp_ratio_mean_cust_var3','cd_sp_ratio_sum_cust_var3'
     ]
cust_coup_hist3=pd.DataFrame(np.row_stack(data_outputs),columns=cols)

CPU times: user 1.1 s, sys: 176 ms, total: 1.28 s
Wall time: 4min 46s


In [16]:
cust_coup_hist3.head()

,campaign_id,customer_id,num_bought_cust_var3,num_discount_availed_cust_var3,frac_discount_availed_cust_var3,sum_discount_cust_var3,selling_price_sum_cust_var3,coupon_discount_mean_cust_var3,selling_price_mean_cust_var3,other_discount_sum_cust_var3,other_discount_mean_cust_var3,cd_sp_ratio_mean_cust_var3,cd_sp_ratio_sum_cust_var3
0,13.0,1053.0,32.0,0.0,0.000000,0.00,2811.83,0.000000,87.869687,-1145.51,-35.797187,0.00000,0.000000
1,13.0,48.0,0.0,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.00000,0.000000
2,9.0,205.0,22.0,3.0,0.136364,-78.36,2670.08,-3.561818,121.367273,-708.13,-32.187727,-0.02365,-0.520295
3,13.0,1050.0,12.0,0.0,0.000000,0.00,983.83,0.000000,81.985833,-377.58,-31.465000,0.00000,0.000000
4,8.0,1489.0,6.0,0.0,0.000000,0.00,284.60,0.000000,47.433333,-80.14,-13.356667,0.00000,0.000000


In [17]:
cust_coup_hist3.to_csv('../cust_coup_hist3_v2.csv',index=False)

In [18]:
%%time
def get_feat(row):
    data_=row[1]
    temp=transaction[(transaction.item_id.isin(get_items_for_coupons(get_coupons_for_customer(data_.customer_id))))&\
               (transaction.date<data_.start_date)]
    if temp.shape[0]>0:
        a=temp['coupon_discount']/temp['selling_price']
        feats=[data_['campaign_id'],data_['customer_id']]+[temp.shape[0],   sum(temp['coupon_discount']<0),  sum(temp['coupon_discount']<0)/temp.shape[0],\
                          temp['coupon_discount'].sum(),temp['selling_price'].sum(),
                          
                          temp['coupon_discount'].mean(),temp['selling_price'].mean(),temp['other_discount'].sum(),\
                          temp['other_discount'].mean(),a.mean(),a.sum() 
                         ]
    else:
        feats=[data_['campaign_id'],data_['customer_id']]+np.repeat(0,11).tolist()

    return feats
temp=df[['customer_id','start_date','campaign_id']].drop_duplicates()
try:
    pool = Pool(16) 
    data_outputs = pool.map(get_feat, temp.iterrows())
finally: 
    pool.close()
    pool.join()
    
cols=['campaign_id','customer_id','num_bought_cust_var2','num_discount_availed_cust_var2','frac_discount_availed_cust_var2','sum_discount_cust_var2',
'selling_price_sum_cust_var2',
'coupon_discount_mean_cust_var2','selling_price_mean_cust_var2','other_discount_sum_cust_var2',
      'other_discount_mean_cust_var2','cd_sp_ratio_mean_cust_var2','cd_sp_ratio_sum_cust_var2'
     ]
cust_coup_hist2=pd.DataFrame(np.row_stack(data_outputs),columns=cols)

CPU times: user 1.12 s, sys: 252 ms, total: 1.38 s
Wall time: 5min 28s


In [19]:
cust_coup_hist2.head()

,campaign_id,customer_id,num_bought_cust_var2,num_discount_availed_cust_var2,frac_discount_availed_cust_var2,sum_discount_cust_var2,selling_price_sum_cust_var2,coupon_discount_mean_cust_var2,selling_price_mean_cust_var2,other_discount_sum_cust_var2,other_discount_mean_cust_var2,cd_sp_ratio_mean_cust_var2,cd_sp_ratio_sum_cust_var2
0,13.0,1053.0,217684.0,5062.0,0.023254,-192327.19,26647765.59,-0.883516,122.414902,-5117819.62,-23.510316,-0.008436,-1836.363787
1,13.0,48.0,158027.0,3048.0,0.019288,-121908.79,17049927.96,-0.771443,107.892499,-4793222.58,-30.331669,-0.006804,-1075.152141
2,9.0,205.0,268278.0,4418.0,0.016468,-148627.81,30950915.23,-0.554007,115.368816,-6804187.94,-25.362452,-0.005450,-1462.171739
3,13.0,1050.0,237809.0,2315.0,0.009735,-80520.71,25406567.05,-0.338594,106.836020,-5212278.18,-21.917918,-0.004414,-1049.704344
4,8.0,1489.0,82673.0,1033.0,0.012495,-31867.67,11912258.86,-0.385466,144.088866,-2742266.67,-33.170039,-0.003423,-283.010166


In [20]:
cust_coup_hist.to_csv('../customer_coupon_var1_v2.csv',index=False)
cust_coup_hist2.to_csv('../customer_coupon_var2_v2.csv',index=False)